In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [6]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [25]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [8]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [9]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [27]:
##########################################
# --- Agente 1: Coletor de Informações --- #
##########################################
from google.adk.agents import Agent
from google.genai import types
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from datetime import date

def agente_coletor():
    coletor = Agent(
        name="agente_coletor",
        model="gemini-2.0-flash",
        instruction="Você é um assistente que coleta informações para criar mensagens de aniversário. Peça detalhes adicionais ao usuário sobre cada aniversariante.",
        description="Agente que coleta informações de aniversários."
    )
    return coletor

def coletar_informacoes_manualmente():
    aniversariantes = []
    while True:
        nome = input("Digite o nome do aniversariante (ou 'fim' para encerrar): ")
        if nome.lower() == 'fim':
            break
        aniversariantes.append(nome)
    return aniversariantes

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    print(f"DEBUG: call_agent chamado para o agente '{agent.name}' com a mensagem: '{message_text}'")  # DEBUG

    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        print(f"DEBUG: call_agent recebeu evento: {event}")  # DEBUG
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"

    print(f"DEBUG: call_agent retornando: '{final_response}'")  # DEBUG
    return final_response

# --- Fluxo Principal ---

print("🎉 Gerador de Mensagens de Aniversário 🎉")

aniversariantes = coletar_informacoes_manualmente()

if aniversariantes:
    print("\nAniversariantes:")
    for aniversariante in aniversariantes:
        print(f"- {aniversariante}")

    coletor = agente_coletor()
    informacoes_coletadas = ""
    for aniversariante in aniversariantes:
        print(f"DEBUG: Chamando call_agent para {aniversariante}")  # DEBUG
        detalhes = call_agent(coletor, f"Informações sobre {aniversariante}: ")
        print(f"DEBUG: call_agent retornou para {aniversariante}: '{detalhes}'")  # DEBUG
        informacoes_coletadas += f"\n{aniversariante}: {detalhes}"

    print("\n--- 📝 Informações Coletadas ---\n")
    print(informacoes_coletadas)

    # ... (Resto do seu código para gerar a mensagem)

else:
    print("Nenhum aniversariante inserido.")

🎉 Gerador de Mensagens de Aniversário 🎉
Digite o nome do aniversariante (ou 'fim' para encerrar): ruthe
Digite o nome do aniversariante (ou 'fim' para encerrar): fim

Aniversariantes:
- ruthe
DEBUG: Chamando call_agent para ruthe
DEBUG: call_agent chamado para o agente 'agente_coletor' com a mensagem: 'Informações sobre ruthe: '
DEBUG: call_agent recebeu evento: content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='Para criar uma mensagem de aniversário especial para Ruthe, preciso de mais algumas informações. Você poderia me dizer:\n\n*   **O que Ruthe significa para você?** (Amiga, familiar, colega de trabalho, etc.)\n*   **Quais são os interesses ou hobbies dela?** (Livros, esportes, viagens, culinária, etc.)\n*   **Existe alguma memória ou momento especial que vocês compartilharam?**\n*   **Qual é o tipo de mensagem que você gostaria de envi

In [14]:
######################################
# --- Agente 3: Redator aniversario --- #
######################################
def agente_redator_aniversario():
    redator = Agent(
        name="agente_redator_aniversario",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar mensagens de aniversário criativas e personalizadas.
            Use as informações fornecidas para gerar uma mensagem única.
            """,
        description="Agente que gera mensagens de aniversário."
    )
    return redator

In [28]:
from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🎉 Gerador de Mensagens de Aniversário 🎉")

# --- Obter as Informações do Usuário ---
nome_aniversariante = input("🎂  Digite o nome do aniversariante: ")
detalhes_aniversariante = input(
    "✍️  Forneça detalhes sobre o aniversariante (personalidade, interesses, etc.): "
)
# Inserir lógica do sistema de agentes ################################################
if not nome_aniversariante:
    print("Você esqueceu de digitar o nome do aniversariante!")
else:
    print(f"✨  Preparando mensagem para {nome_aniversariante}! ✨")

    informacoes = f"Nome: {nome_aniversariante}\\nDetalhes: {detalhes_aniversariante}\\nData: {data_de_hoje}"

# Agente Redator (agora ele faz o trabalho do coletor também)
redator = agente_redator_aniversario()
mensagem_gerada = call_agent(redator, informacoes)
print("\n--- 📝 Mensagem Gerada ---\n")
print(mensagem_gerada)
print("--------------------------------------------------------------")

# (Opcional) Agente Revisor
#revisor = agente_revisor_aniversario()
#mensagem_revisada = call_agent(revisor, mensagem_gerada)
#print("\n--- 📝 Mensagem Revisada ---\n")
#print(mensagem_revisada)
#print("--------------------------------------------------------------")

print("\n🎉 Mensagem de aniversário pronta! 🎉")

🎉 Gerador de Mensagens de Aniversário 🎉
🎂  Digite o nome do aniversariante: ruthe
✍️  Forneça detalhes sobre o aniversariante (personalidade, interesses, etc.): ela é simpatica e gosta de bike
✨  Preparando mensagem para ruthe! ✨
DEBUG: call_agent chamado para o agente 'agente_redator_aniversario' com a mensagem: 'Nome: ruthe\nDetalhes: ela é simpatica e gosta de bike\nData: 17/05/2025'
DEBUG: call_agent recebeu evento: content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='## Sugestão de Mensagem de Aniversário para Ruthe:\n\n**Opção 1: Aventurosa e Energética**\n\nRuthe, que a sua pedalada pela vida continue sempre com o vento a favor e o sol no rosto! Que este novo ano traga trilhas incríveis, paisagens deslumbrantes e muita energia para conquistar todos os seus objetivos. Feliz aniversário! 🚴\u200d♀️☀️\n\n**Opção 2: Simples e Carinhosa**\n\nR